# Create a Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import max, min, avg, col, count, date_format

# Create a SparkSession
spark = SparkSession.builder.appName("UberDataAnalysis").getOrCreate()

# Load the dataset into a DataFrame
uber_df = spark.read.csv("uber.csv", header=True, inferSchema=True)
uber_df.show(5)  # Display the first 5 rows of the DataFrame

+---------+-----------+--------+------+---------------+--------+--------------+
|     Date|Time(Local)|Eyeballs|Zeroes|Completed_Trips|Requests|Unique_Drivers|
+---------+-----------+--------+------+---------------+--------+--------------+
|10-Sep-12|          7|       5|     0|              2|       2|             9|
|10-Sep-12|          8|       6|     0|              2|       2|            14|
|10-Sep-12|          9|       8|     3|              0|       0|            14|
|10-Sep-12|         10|       9|     2|              0|       1|            14|
|10-Sep-12|         11|      11|     1|              4|       4|            11|
+---------+-----------+--------+------+---------------+--------+--------------+
only showing top 5 rows



# Which date had most completed trips during 3 week period

In [2]:
completed_trips=uber_df.groupBy("Date").sum("Completed_Trips")
completed_trips.show()

most_completed_trips=completed_trips.orderBy("sum(Completed_Trips)",ascending=False).select("Date").first()["Date"]
print(most_completed_trips)

+---------+--------------------+
|     Date|sum(Completed_Trips)|
+---------+--------------------+
|11-Sep-12|                  40|
|13-Sep-12|                  47|
|18-Sep-12|                  42|
|21-Sep-12|                 190|
|12-Sep-12|                  91|
|19-Sep-12|                  41|
|14-Sep-12|                 305|
|16-Sep-12|                 150|
|23-Sep-12|                 111|
|24-Sep-12|                   4|
|22-Sep-12|                 248|
|20-Sep-12|                  70|
|10-Sep-12|                  26|
+---------+--------------------+

14-Sep-12


# What was the highest number of completed trips within a 24-hour period?

In [19]:
#Group the data by 24-hour window
from pyspark.sql.functions import to_timestamp, hour, window

# Corrected the column name from 'Time (Local)' to 'Time(Local)'
uber_df_updated = uber_df.withColumn(
    'Time(Local)',
    to_timestamp(to_date(col('Date'),'dd-MMM-yy')) + make_interval(hours=hour(col('Time(Local)')))
)
uber_df_updated.show()
# Corrected column name in window function to 'Time(Local)'
completed_trips_by_window = uber_df_updated \
    .groupBy(window("Time(Local)", "24 hours")) \
    .agg(sum("Completed_Trips").alias("Total Completed Trips")) \
    .orderBy("Total Completed Trips", ascending=False)

highest_completed_trips_in_24_hours = completed_trips_by_window \
    .select("Total Completed Trips") \
    .first()["Total Completed Trips"]

print(highest_completed_trips_in_24_hours)

+---------+-------------------+--------+------+---------------+--------+--------------+
|     Date|        Time(Local)|Eyeballs|Zeroes|Completed_Trips|Requests|Unique_Drivers|
+---------+-------------------+--------+------+---------------+--------+--------------+
|10-Sep-12|2012-09-10 07:00:00|       5|     0|              2|       2|             9|
|10-Sep-12|2012-09-10 08:00:00|       6|     0|              2|       2|            14|
|10-Sep-12|2012-09-10 09:00:00|       8|     3|              0|       0|            14|
|10-Sep-12|2012-09-10 10:00:00|       9|     2|              0|       1|            14|
|10-Sep-12|2012-09-10 11:00:00|      11|     1|              4|       4|            11|
|10-Sep-12|2012-09-10 12:00:00|      12|     0|              2|       2|            11|
|10-Sep-12|2012-09-10 13:00:00|       9|     1|              0|       0|             9|
|10-Sep-12|2012-09-10 14:00:00|      12|     1|              0|       0|             9|
|10-Sep-12|2012-09-10 15:00:00| 

#Which hour of the day had the most requests during the two-week period?

In [24]:
hourly_requests = uber_df_updated.groupBy(hour("Time(Local)").alias("hour")).agg(sum("Requests").alias("total_requests")).orderBy("total_requests", ascending=False)
most_requested_hour = hourly_requests.select("hour").first()[0]
print("The hour with the most requests is:", most_requested_hour)

The hour with the most requests is: 23


#What percentages of all zeroes during the two-week period occurred on weekends (Friday at 5 pm to Sunday at 3 am)?

In [32]:
from pyspark.sql.functions import col, hour, dayofweek, when, to_date

# Cast 'Date' column to date type
uber_df_updated = uber_df_updated.withColumn("Date", to_date(col("Date"), "dd-MMM-yy"))

weekend_zeros = uber_df_updated.filter(
    (
        (dayofweek(col("Date")) == 6) & (hour(col("Time(Local)")) >= 17)  # Friday 5 pm onwards
        | (dayofweek(col("Date")) == 7)  # Saturday all day
        | ((dayofweek(col("Date")) == 1) & (hour(col("Time(Local)")) < 3))  # Sunday until 3 am
    )
).agg(
    sum(when(col("Zeroes").isNotNull(), col("Zeroes")).otherwise(0)).alias("weekend_zeros")
).collect()[0]["weekend_zeros"]

total_zeros = uber_df_updated.agg(sum("Zeroes").alias("total_zeros")).collect()[0]["total_zeros"]

percent_weekend_zeros = (weekend_zeros / total_zeros) * 100 if total_zeros else 0  # Handle division by zero

print("The percentage of zeros that occurred on weekends is:", percent_weekend_zeros, "%")

The percentage of zeros that occurred on weekends is: 38.20853743876837 %
